In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import shutil
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [2]:
# Listing all files
os.listdir('/kaggle/input/camp-qmind-hackathon-2023-B')

['sample_submission.csv', 'train.csv', 'test.csv']

In [3]:
train_df = pd.read_csv('/kaggle/input/camp-qmind-hackathon-2023-B/train.csv')
train_df.head()  # An overview of the training set

,ID,sepal_length,sepal_width,petal_length,petal_width,species
0,31,6.1,3.0,4.6,1.4,b
1,32,7.7,3.0,6.1,2.3,c
2,33,5.6,2.5,3.9,1.1,b
3,34,6.4,2.8,5.6,2.1,c
4,35,5.8,2.8,5.1,2.4,c


# Create Your Model

In [27]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [29]:
from sklearn.preprocessing import StandardScaler, LabelEncoder
# define your model, train, and make predictions on the test set
from sklearn.model_selection import train_test_split
X = train_df[['sepal_length','sepal_width','petal_length','petal_width']]
y = train_df['species']
encoder = LabelEncoder()
y = encoder.fit_transform(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [37]:
model = keras.Sequential([
    layers.Input(shape=(4,)),          # Input layer with 4 features
    layers.Dense(64, activation='relu'),  # Hidden layer with 64 units and ReLU activation
    layers.Dense(32, activation='relu'),  # Hidden layer with 32 units and ReLU activation
    layers.Dense(3, activation='softmax')  # Output layer with 3 units for 3 flower species and softmax activation
])

model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=50, validation_split=0.2)
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

new_data = scaler.transform(X_test)  # Preprocess new data if needed
predictions = model.predict(X_test)
print(predictions)




Epoch 1/100
2/2 [==============================] - 1s 206ms/step - loss: 1.2082 - accuracy: 0.0448 - val_loss: 1.2200 - val_accuracy: 0.2353
Epoch 2/100
2/2 [==============================] - 0s 34ms/step - loss: 1.1463 - accuracy: 0.1343 - val_loss: 1.1714 - val_accuracy: 0.4118
Epoch 3/100
2/2 [==============================] - 0s 33ms/step - loss: 1.0903 - accuracy: 0.3881 - val_loss: 1.1250 - val_accuracy: 0.4118
Epoch 4/100
2/2 [==============================] - 0s 32ms/step - loss: 1.0338 - accuracy: 0.5075 - val_loss: 1.0811 - val_accuracy: 0.4706
Epoch 5/100
2/2 [==============================] - 0s 32ms/step - loss: 0.9863 - accuracy: 0.5970 - val_loss: 1.0413 - val_accuracy: 0.4706
Epoch 6/100
2/2 [==============================] - 0s 33ms/step - loss: 0.9405 - accuracy: 0.7164 - val_loss: 1.0057 - val_accuracy: 0.4706
Epoch 7/100
2/2 [==============================] - 0s 34ms/step - loss: 0.9022 - accuracy: 0.7164 - val_loss: 0.9738 - val_accuracy: 0.5882
Epoch 8/100
2/2 [==

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [8]:
# define your model, train, and make predictions on the test set
from sklearn.model_selection import train_test_split
X = train_df[['sepal_length','sepal_width','petal_length','petal_width']]
y = train_df['species']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

randomForest = RandomForestClassifier(n_estimators=1,random_state=42)
randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)
print(y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report with additional metrics
print(classification_report(y_test, y_pred))

['b' 'b' 'c' 'c' 'c' 'c' 'b' 'c' 'b' 'b' 'a' 'c' 'a' 'a' 'c' 'c' 'a' 'b'
 'c' 'b' 'c' 'b' 'c' 'a' 'c' 'b' 'c' 'b' 'b' 'a' 'c' 'a' 'c' 'b' 'c' 'a']
Accuracy: 0.9444444444444444
              precision    recall  f1-score   support

           a       1.00      1.00      1.00         8
           b       0.92      0.92      0.92        12
           c       0.94      0.94      0.94        16

    accuracy                           0.94        36
   macro avg       0.95      0.95      0.95        36
weighted avg       0.94      0.94      0.94        36



In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
randomForest = RandomForestClassifier()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=randomForest, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
grid_search_results = grid_search.cv_results_

print(grid_search_results)


In [15]:
print(grid_search_results)

{'mean_fit_time': array([0.3386857 , 0.68752713, 0.96543565, 0.31361923, 0.61973653,
       0.94046059, 0.3204164 , 0.63875175, 0.94336739, 0.32059326,
       0.61799169, 0.96014943, 0.30962601, 0.61952062, 0.93266077,
       0.34378462, 0.86746459, 1.04721642, 0.3152709 , 0.63813066,
       0.95118103, 0.32029672, 0.63252273, 0.97123513, 0.31448092,
       0.63086319, 0.95433912, 0.31793356, 0.63653383, 0.93519492,
       0.32118111, 0.61021485, 0.9203681 , 0.31292071, 0.62301941,
       0.96468482, 0.31078877, 0.61979957, 0.92222743, 0.31210089,
       0.61675291, 0.91000056, 0.30718241, 0.61357002, 0.93424058,
       0.3188262 , 0.61653948, 0.96051707, 0.31185455, 0.62382135,
       0.92394161, 0.31314535, 0.62284007, 0.92874017, 0.31459398,
       0.83696213, 1.04574695, 0.3115068 , 0.6108613 , 0.96601005,
       0.31117177, 0.62273307, 0.92309141, 0.31213293, 0.6230885 ,
       0.92462492, 0.31479125, 0.62531877, 0.94089651, 0.31584649,
       0.62906313, 0.97866564, 0.31095343, 0

In [14]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Generate a classification report with additional metrics
print(classification_report(y_test, y_pred))

Accuracy: 0.8611111111111112
              precision    recall  f1-score   support

           a       1.00      1.00      1.00         8
           b       0.73      0.92      0.81        12
           c       0.92      0.75      0.83        16

    accuracy                           0.86        36
   macro avg       0.89      0.89      0.88        36
weighted avg       0.88      0.86      0.86        36



# Prepare a CSV for Submission

In [ ]:
# Reading in the test dataset
test_df = pd.read_csv('/kaggle/input/camp-qmind-hackathon-2023-B/test.csv')
test_ids = test_df['ID']
test_df.head() # Identical format to the train data, without a class label (your model will predict this)

In [ ]:
# When you have the ids and predictions prepared, here is how you make a submission.csv
predicted_class = [0] * 30  #TODO: replace this array with your own predictions. These are placeholder predictions.

submission = {"ID": np.array(test_ids).astype(int), "species": predicted_class}
submisson_df = pd.DataFrame(submission)

# you can sort the data frame by ID (you don't have to)
submisson_df = submisson_df.sort_values(by=['ID'])
print(submisson_df)
submisson_df.to_csv('/kaggle/working/submission.csv', index=False)